In [1]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import gurobipy_pandas as gppd

# set intercative mode
gppd.set_interactive()

In [2]:
# importing data
hubs = pd.read_csv(r"D:\1. UTK PhD\Fall 23\IE 522\Final Project\OneDrive_1_12-5-2023\TX_hubs.csv")
network = pd.read_csv(r"D:\1. UTK PhD\Fall 23\IE 522\Final Project\OneDrive_1_12-5-2023\TX_network.csv")
plants = pd.read_csv(r"D:\1. UTK PhD\Fall 23\IE 522\Final Project\OneDrive_1_12-5-2023\TX_plants.csv")
roads = pd.read_csv(r"D:\1. UTK PhD\Fall 23\IE 522\Final Project\OneDrive_1_12-5-2023\TX_roads.csv")
suppliers = pd.read_csv(r"D:\1. UTK PhD\Fall 23\IE 522\Final Project\OneDrive_1_12-5-2023\TX_suppliers.csv")
railroads = pd.read_csv(r"D:\1. UTK PhD\Fall 23\IE 522\Final Project\OneDrive_1_12-5-2023\TX_railroads.csv")

In [3]:
hubs.set_index('hub', inplace=True)
hubs.head()

,index,latitude,longitude,invest,capacity
hub,,,,,
17201,1,33.64844,-95.56841,3476219,300000
17218,2,33.64124,-96.60666,3476219,300000
17359,3,29.31990,-96.10283,3476219,300000
17372,4,30.75623,-98.67770,3476219,300000
17395,5,30.84636,-96.98711,3476219,300000


In [4]:
network.head()

,counties,hubs,plants,techs,demand
0,254,33,167,1,1476310602


In [5]:
plants.set_index('plant', inplace=True)
plants.head()

,index,latitude,longitude,tech,invest,capacity,yield
plant,,,,,,,
541,1,30.220353,-97.740436,1,130956797,152063705,232
542,2,29.676724,-98.635234,1,130956797,152063705,232
543,3,33.760315,-96.559867,1,130956797,152063705,232
544,4,29.531920,-98.286901,1,130956797,152063705,232
545,5,29.688568,-98.562424,1,130956797,152063705,232


In [6]:
roads.set_index(['county','hub'], inplace=True)
roads.head()

index  distance       cost
county hub                              
48001  17201      1  219.6090  40.580691
       17218      2  264.7658  47.661277
       17359      3  370.6750  64.267840
       17372      4  420.0082  72.003286
       17395      5  199.2915  37.394907

In [7]:
# cleaning dataset by removing NA values
suppliers.dropna(inplace=True)
suppliers.county = suppliers.county.astype(int)
suppliers.set_index('county',inplace=True)
suppliers.head()

,index,supply
county,,
48001,1.0,13131.971710
48003,2.0,1177.351950
48005,3.0,3854.618542
48007,4.0,308.182629
48009,5.0,19802.136510


In [8]:
railroads.set_index(['hub','plant'], inplace=True)
railroads.head()

index     distance       cost  loading  capacity
hub   plant                                                  
17201 541        1   922.023251  17.095711  3066792    338000
      542        2  1160.842967  20.923752  3066792    338000
      543        3   186.924258   5.312809  3066792    338000
      544        4  1074.699073  19.542951  3066792    338000
      545        5  1160.842967  20.923752  3066792    338000

In [9]:
# create gurobi model
m = gp.Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-17


In [10]:
### Variables
yhub = gppd.add_vars(m, hubs, name='yhub', vtype=GRB.BINARY, lb=0)
yplant = gppd.add_vars(m, plants, name='yplant', vtype=GRB.BINARY, lb=0)
ytruck = gppd.add_vars(m, roads, name='ytruck', vtype=GRB.BINARY, lb=0)
ytrain = gppd.add_vars(m, railroads, name='ytrain', vtype=GRB.BINARY, lb=0)

c_to_h = gppd.add_vars(m, roads, name='c_to_h', vtype=GRB.CONTINUOUS, lb=0) # Flow from County to hub
h_to_p = gppd.add_vars(m, railroads, name='h_to_p', vtype=GRB.CONTINUOUS, lb=0) # Flow from hub to plant

x_out = m.addVar(vtype=GRB.CONTINUOUS, name='outside_supply', lb=0)

In [11]:
### parameters
"""
demand = network['demand]
yield = plants['yield']
cplant = plants['invest']
chub = hubs['invest']
"""
M = 500000
# fuel from external source

In [12]:
### Objective
# m.setObjective((plants['invest'] * yplant).sum() + (hubs['invest'] * yhub).sum() + (ytruck * c_to_h * roads['cost']).sum() \
#                + (ytrain * railroads['cost']).sum() + (ytrain * h_to_p * railroads['loading']).sum(), 
#                GRB.MINIMIZE)

In [13]:
### Highest possible demand that can be met by the network
m.setObjective((yplant * plants['yield'] * h_to_p.groupby('plant').sum()).sum(), GRB.MAXIMIZE)

In [14]:
### Constraints
# flow conservation through hub
flow_hub = gppd.add_constrs(m,
                            c_to_h.groupby('hub').sum(),
                            GRB.EQUAL,
                            h_to_p.groupby('hub').sum(),
                            name='hub_flow'
                            )

In [15]:
# hub capacity
cap_hub = gppd.add_constrs(m,
                           c_to_h.groupby('hub').sum(),
                           GRB.LESS_EQUAL,
                           hubs['capacity']*yhub,
                           name='hub_capacity'
                           )

In [16]:
# train capacity
cap_train = gppd.add_constrs(m,
                           h_to_p,
                           GRB.LESS_EQUAL,
                           railroads['capacity']*ytrain,
                           name='train_capacity'
                           )

In [17]:
# supplier (county) capacity
cap_county = gppd.add_constrs(m,
                           (c_to_h).groupby('county').sum(),
                           GRB.LESS_EQUAL,
                           suppliers['supply'],
                           name='suppliers_supply'
                           )

In [18]:
# delivery by truck
truck_delivery = gppd.add_constrs(m,
                           c_to_h,
                           GRB.LESS_EQUAL,
                           M*ytruck,
                           name='truck_binary'
                           )

# delivery by train
cap_train = gppd.add_constrs(m,
                           h_to_p,
                           GRB.LESS_EQUAL,
                           M*ytrain,
                           name='train_binary'
                           )

In [19]:
# plant capacity
cap_plant = gppd.add_constrs(m,
                           plants['yield'] * h_to_p.groupby('plant').sum(),
                           GRB.LESS_EQUAL,
                           plants['capacity']*yplant,
                           name='plant_capacity'
                           )

In [20]:
# demand
# demand_constraint = gppd.add_constrs(m,
#                                      (yplant * plants['yield'] * h_to_p.groupby('plant').sum()).sum(),
#                                      GRB.GREATER_EQUAL,
#                                      network['demand']*0.45,
#                                      name='network_demand'
#                                      )

In [21]:
# write the model
m.write("supply_chain_biofuel.lp")

In [22]:
m.params.LogToConsole = 1
#m.params.MIPgap = 0.05
m.params.NumericFocus = 2

m.update()
m.optimize()

Set parameter NumericFocus to value 2
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 19891 rows, 27987 columns and 75176 nonzeros
Model fingerprint: 0xbafa7dad
Model has 5511 quadratic objective terms
Variable types: 13894 continuous, 14093 integer (14093 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+08]
  Objective range  [0e+00, 0e+00]
  QObjective range [5e+02, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e-01, 5e+04]
Found heuristic solution: objective -0.0000000
Presolve removed 19891 rows and 27987 columns
Presolve time: 0.02s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.05 seconds (0.04 work units)
Thread count was 1 (of 20 available processors)

Solution count 2: 7.08384e+08 -0 

Optimal solution found (tolerance 1.

In [ ]:
# m.computeIIS()

In [25]:
m.getObjective().getValue()

708383628.3169078

In [22]:
hub_to_plant = [m.getVarByName(f"h_to_p[{i},{j}]").X for i in hubs.index for j in plants.index]
county_to_hub = [m.getVarByName(f"c_to_h[{i},{j}]").X for i in suppliers.index for j in hubs.index]
yplant = [m.getVarByName(f"yplant[{i}]").X for i in plants.index]
yhub = [m.getVarByName(f"yhub[{i}]").X for i in hubs.index]
ytrain = [m.getVarByName(f"ytrain[{i},{j}]").X for i in hubs.index for j in plants.index]
ytruck = [m.getVarByName(f"ytruck[{i},{j}]").X for i in suppliers.index for j in hubs.index]

In [23]:
sum(ytruck)

8184.0

In [24]:
sum(county_to_hub)

2863533.495258799

In [27]:
sum(hub_to_plant)*232

664339770.9000001

In [26]:
suppliers['supply'].sum()*232

708383628.3169292

In [50]:
np.nonzero(hub_to_plant)

(array([ 304,  468, 1059, 2002, 2475, 2806, 3143, 3672, 4309, 4410, 4674,
        5209, 5314], dtype=int64),)

In [24]:
np.nonzero(county_to_hub)

(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
        143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
        156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
        169, 170, 171, 172, 173, 174, 175, 176, 177

In [29]:
roads_copy = roads.copy()
roads_copy['c_to_h'] = c_to_h.gppd.X.to_frame()
roads_copy = roads_copy.groupby('county').sum()

In [30]:
roads_copy['supply'] = suppliers['supply'].copy()

In [36]:
vio_list = []
for index, row in roads_copy.iterrows():
    if row['c_to_h'] > row['supply']+0.0001:
        vio_list.append(index)

In [37]:
roads_copy.loc[vio_list]

,index,distance,cost,c_to_h,supply
county,,,,,


In [27]:
sum(hub_to_plant)*232

738155301.0

In [28]:
sum(yhub)

11.0

In [29]:
yhub

[-0.0,
 1.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 1.0,
 1.0,
 -0.0,
 1.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 1.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 1.0,
 -0.0]

In [30]:
sum(yplant)

6.0